In [1]:
import os
import sys
sys.path.append(os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath("")))))

In [2]:
from rmg_gua.gua_peuqse.runtime_utilities import get_all_param_lists, make_exp_data_lists
from rmg_gua.gua_peuqse.ct_simulation import simulationFunction

/work/westgroup/ChrisB/_01_MeOH_repos/uncertainty_analysis


In [3]:
os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath(""))))

'/work/westgroup/ChrisB/_01_MeOH_repos/uncertainty_analysis'

In [ ]:
# modify one of the parameters (the reaction that makes methanol)


In [4]:
param_list = get_all_param_lists("./config")
for key in param_list.keys():
    print(len(param_list[key]))

110
110
110
110
110


In [5]:
x,xa, y,yu = make_exp_data_lists("../../gua_cantera", "./config", use_count=4)

In [6]:
test_sbr, ydata = simulationFunction(param_list["value_list"], debug=True)

input parameters:  [17.621176281775035, 129300.00000000001, 0.94, 17.621176281775035, 106100.0, 0.68, 17.621176281775035, 106100.0, 0.68, 17.621176281775035, 1900.0, 0.02, 17.143951116423963, 106100.0, 0.68, 17.143951116423963, 106100.0, 0.68, 17.621176281775035, 129300.00000000001, 0.94, 17.621176281775035, 129300.00000000001, 0.94, 17.621176281775035, 15400.0, 0.65, 17.143951116423963, 129300.00000000001, 0.94, 17.143951116423963, 129300.00000000001, 0.94, 13.0, 0.0, 0.5, 13.0, 0.0, 0.5, 13.0, 20920.0, 0.5, 0.1, 3800.0, 0.0, 0.17897694729316943, 58000.0, 0.0, 0.1, 107900.0, 0.69, 0.1, 107900.0, 0.69, 0.033, 107100.0, 0.76, 0.01, 41840.0, 0.0, 0.1, 0.0, 0.0, 17.621176281775035, 185142.0, 0.84, 17.621176281775035, 75300.0, 0.57, 17.621176281775035, 73300.0, 0.26, 17.621176281775035, 117700.0, 0.58, 13.0, 0.0, 0.44, 17.621176281775035, 185100.0, 0.84, 17.320146286111054, 97500.0, 0.51, 17.621176281775035, 117700.0, 0.58, 17.621176281775035, 75250.0, 0.57, 17.143014800254097, 75250.0, 0.

/work/westgroup/ChrisB/_01_MeOH_repos/uncertainty_analysis/rmg_gua/gua_cantera/Spinning_basket_reactor/sbr.py:93: UserWarning: InterfaceReaction2::validate: 
 Sticking coefficient is greater than 1 for reaction 'C(40) + 2 X(1) <=> CH3X(55) + H*(10)'
 at T = 10000.0

  self.surf = ct.Interface(yaml_file, "surface1", [self.gas])


changing reactions
X(1)
starting sim 1
changing reactions
X(1)
starting sim 2
changing reactions
X(1)
starting sim 3
changing reactions
X(1)
Results:

CH3OH:  [2.6472940871847663e-08, 1.4761354823037989e-08, 2.978019256460928e-08, 5.019339263200844e-08]
CO:  [0.09512979866411266, 0.0964904328480798, 0.0958524836027715, 0.09457922123179276]
CO2:  [0.15019273742932807, 0.14515626509712407, 0.1475428270989096, 0.15223491898933003]
H2:  [0.059847954014873674, 0.032847500889377976, 0.044922444964174434, 0.07073677506757514]
H2O:  [0.4631910025634246, 0.4835919547397325, 0.4744358131285233, 0.45495682233963414]
sim done
start time: Tue Mar 21 09:21:11 2023
end time: Tue Mar 21 09:21:16 2023
elapsed: 5.400837182998657 seconds


In [ ]:
import copy
import pickle
import shutil
import os
import math
import yaml

import numpy as np
import PEUQSE as PEUQSE
import PEUQSE.UserInput as UserInput
import sys
repo_dir = os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath(""))))
# repo_dir = os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath(__file__)))))
sys.path.append(repo_dir)
results_path = os.path.join(repo_dir, "rmg_gua", "gua_peuqse", "02_opt_logp_fam_be", "config")
from rmg_gua.gua_peuqse.runtime_utilities import get_all_param_lists, make_exp_data_lists
run_test = False


import ct_simulation  # function for peuquse to optimize

if __name__ == "__main__":
    # Just a simple example. The user can also put the values in directly into the runfile or extract from a csv, for example.
    print("running job")

    results_path = os.path.join(repo_dir, "rmg_gua", "gua_peuqse", "02_opt_logp_fam_be", "config")
    # get the peuqse parameters
    param_dict = get_all_param_lists(results_path=results_path)     

    # load the exp data yaml
    with open("expt_data.yaml", "r") as f:
        expt_data = yaml.load(f, Loader=yaml.FullLoader)
        
    # load uncertainty yaml. experimenting with different layout for X. 
    with open("expt_unc.yaml", "r") as f:
        expt_unc = yaml.load(f, Loader=yaml.FullLoader)
    
    data_path = os.path.join(repo_dir, "rmg_gua", "gua_cantera")
    _, x_data, y_data, y_unc = make_exp_data_lists(data_path, results_path, use_count=3)
    
    x_data = np.array(x_data)
    y_data = np.array(y_data)
    y_unc = np.array(y_data)
    print(f"length is {len(x_data)} in main")

    # Provide the observed X values and Y values and uncertainties -- all should be arrays or lists with nesting like [[1,2,3]] or [[1,2,3,4],[4,5,6,6]]
    UserInput.responses['responses_abscissa'] = x_data
    UserInput.responses['responses_observed'] = y_data

    # estimated graaf uncertainties. 
    UserInput.responses['responses_observed_uncertainties'] = y_unc

    #Optional: provide labels for the responses axes and parameter names.
    UserInput.simulated_response_plot_settings['x_label'] = [
        "catalyst_area", "pressure (Pa)", "mole frac CO", "mole frac CO2", "mole frac H2", "temperature (K)", "volume flowrate (m^3/s)"]
    UserInput.simulated_response_plot_settings['y_label'] = [
        "CH3OH mole Frac out", "CO mole Frac out", "CO2 mole Frac out", "H2 mole Frac out", "H2O mole Frac out"]
    UserInput.model['parameterNamesAndMathTypeExpressionsDict'] = param_dict["label_list"]

    #Provide the prior distribution and uncertainties of the individual parameters.
    # prior expected values for a and b
    UserInput.model['InputParameterPriorValues'] = param_dict["value_list"]
    # required. #If user wants to use a prior with covariance, then this must be a 2D array/ list. To assume no covariance, a 1D
    UserInput.model['InputParametersPriorValuesUncertainties'] = param_dict["unc_list"]
    # UserInput.model['InputParameterInitialGuess'] = #Can optionally change the initial guess to be different from prior means.
    UserInput.model['InputParameterPriorValues_lowerBounds'] = param_dict["lower_list"]
    UserInput.model['InputParameterPriorValues_upperBounds'] = param_dict["upper_list"]
    
    UserInput.model['simulateByInputParametersOnlyFunction'] = ct_simulation.simulation_function_wrapper

    UserInput.parameter_estimation_settings['mcmc_threshold_filter_samples'] = True

    UserInput.parameter_estimation_settings['mcmc_random_seed'] = 0
    UserInput.parameter_estimation_settings['multistart_searchType'] = 'doOptimizeLogP'
    UserInput.parameter_estimation_settings['multistart_passThroughArgs'] = {'method':'BFGS'} #Here, BFGS is used. However, Nelder-Mead is usually what is recommended.
    UserInput.parameter_estimation_settings['multistart_initialPointsDistributionType'] = 'grid'
    UserInput.parameter_estimation_settings['multistart_exportLog'] = True
    
    #After making the UserInput, now we make a 'parameter_estimation' object from it.
    PE_object = PEUQSE.parameter_estimation(UserInput)
    
    # run the model
    PE_object.doMultiStart()
    PE_object.createAllPlots() #This function calls each of the below functions so that the user does not have to.



running job
length is 7 in main
input parameters:  [1.76211763e+01 1.29300000e+05 9.40000000e-01 1.76211763e+01
 1.06100000e+05 6.80000000e-01 1.76211763e+01 1.06100000e+05
 6.80000000e-01 1.76211763e+01 1.90000000e+03 2.00000000e-02
 1.71439511e+01 1.06100000e+05 6.80000000e-01 1.71439511e+01
 1.06100000e+05 6.80000000e-01 1.76211763e+01 1.29300000e+05
 9.40000000e-01 1.76211763e+01 1.29300000e+05 9.40000000e-01
 1.76211763e+01 1.54000000e+04 6.50000000e-01 1.71439511e+01
 1.29300000e+05 9.40000000e-01 1.71439511e+01 1.29300000e+05
 9.40000000e-01 1.30000000e+01 0.00000000e+00 5.00000000e-01
 1.30000000e+01 0.00000000e+00 5.00000000e-01 1.30000000e+01
 2.09200000e+04 5.00000000e-01 1.00000000e-01 3.80000000e+03
 0.00000000e+00 1.78976947e-01 5.80000000e+04 0.00000000e+00
 1.00000000e-01 1.07900000e+05 6.90000000e-01 1.00000000e-01
 1.07900000e+05 6.90000000e-01 3.30000000e-02 1.07100000e+05
 7.60000000e-01 1.00000000e-02 4.18400000e+04 0.00000000e+00
 1.00000000e-01 0.00000000e+00 0.0

/work/westgroup/ChrisB/_01_MeOH_repos/uncertainty_analysis/rmg_gua/gua_cantera/Spinning_basket_reactor/sbr.py:93: UserWarning: InterfaceReaction2::validate: 
 Sticking coefficient is greater than 1 for reaction 'C(40) + 2 X(1) <=> CH3X(55) + H*(10)'
 at T = 10000.0

  self.surf = ct.Interface(yaml_file, "surface1", [self.gas])


changing reactions
X(1)
starting sim 1
changing reactions
X(1)
starting sim 2
changing reactions
X(1)
Results:

CH3OH:  [2.6472940871847663e-08, 1.4761354823037989e-08, 2.978019256460928e-08]
CO:  [0.09512979866411266, 0.0964904328480798, 0.0958524836027715]
CO2:  [0.15019273742932807, 0.14515626509712407, 0.1475428270989096]
H2:  [0.059847954014873674, 0.032847500889377976, 0.044922444964174434]
H2O:  [0.4631910025634246, 0.4835919547397325, 0.4744358131285233]
sim done
start time: Tue Mar 21 09:50:02 2023
end time: Tue Mar 21 09:50:06 2023
elapsed: 3.3533263206481934 seconds
